In [13]:
import json
from pathlib import Path
from datetime import date

from src.penalties import (
    get_duration_factor,
    get_activity_factor,
    get_time_penalty,
)
from src.exposure import compute_exposure_score
from src.windows import generate_time_windows
from src.ranker import label_windows


JSON_PATH = Path("data/delhi_hourly_aqi.json")


with open(JSON_PATH, "r") as f:
    hourly_aqi = json.load(f)["data"]

today = date.today().strftime("%Y-%m-%d")

hourly_aqi = [
    h for h in hourly_aqi
    if h.get("date") == today
]

if not hourly_aqi:
    raise ValueError("No AQI data available for today.")


# User input
duration = 30  # minutes
activity = "walking"

duration_factor = get_duration_factor(duration)
activity_factor = get_activity_factor(activity)

windows = generate_time_windows(hourly_aqi, duration)

In [14]:
current_hour = datetime.now().hour
current_hour

18

In [15]:
windows = [
    w for w in windows
    if w["start_hour"] >= current_hour
]

In [16]:
for w in windows:
    hour_penalties = [get_time_penalty(h) for h in w["hours"]]
    time_penalty = sum(hour_penalties) / len(hour_penalties)
    w["exposure"] = compute_exposure_score(
        w["avg_aqi"],
        duration_factor,
        activity_factor,
        time_penalty,
    )

In [17]:
windows

[{'start_hour': 18,
  'end_hour': 19,
  'avg_aqi': 255.0,
  'hours': [18],
  'exposure': 382.5},
 {'start_hour': 19,
  'end_hour': 20,
  'avg_aqi': 252.0,
  'hours': [19],
  'exposure': 378.0},
 {'start_hour': 20,
  'end_hour': 21,
  'avg_aqi': 249.0,
  'hours': [20],
  'exposure': 373.5},
 {'start_hour': 21,
  'end_hour': 22,
  'avg_aqi': 244.0,
  'hours': [21],
  'exposure': 317.2},
 {'start_hour': 22,
  'end_hour': 23,
  'avg_aqi': 239.0,
  'hours': [22],
  'exposure': 310.7},
 {'start_hour': 23,
  'end_hour': 24,
  'avg_aqi': 233.0,
  'hours': [23],
  'exposure': 302.90000000000003}]

In [18]:
max_exposure = max(w["exposure"] for w in windows)

for w in windows:
    w["NormalizedScore"] = round(
        (w["exposure"] / max_exposure) * 100, 1
    )

In [19]:
windows

[{'start_hour': 18,
  'end_hour': 19,
  'avg_aqi': 255.0,
  'hours': [18],
  'exposure': 382.5,
  'NormalizedScore': 100.0},
 {'start_hour': 19,
  'end_hour': 20,
  'avg_aqi': 252.0,
  'hours': [19],
  'exposure': 378.0,
  'NormalizedScore': 98.8},
 {'start_hour': 20,
  'end_hour': 21,
  'avg_aqi': 249.0,
  'hours': [20],
  'exposure': 373.5,
  'NormalizedScore': 97.6},
 {'start_hour': 21,
  'end_hour': 22,
  'avg_aqi': 244.0,
  'hours': [21],
  'exposure': 317.2,
  'NormalizedScore': 82.9},
 {'start_hour': 22,
  'end_hour': 23,
  'avg_aqi': 239.0,
  'hours': [22],
  'exposure': 310.7,
  'NormalizedScore': 81.2},
 {'start_hour': 23,
  'end_hour': 24,
  'avg_aqi': 233.0,
  'hours': [23],
  'exposure': 302.90000000000003,
  'NormalizedScore': 79.2}]

In [20]:
windows = sorted(windows, key=lambda x: x["exposure"])
windows = label_windows(windows)

In [21]:
windows

[{'start_hour': 23,
  'end_hour': 24,
  'avg_aqi': 233.0,
  'hours': [23],
  'exposure': 302.90000000000003,
  'NormalizedScore': 79.2,
  'label': 'Best'},
 {'start_hour': 22,
  'end_hour': 23,
  'avg_aqi': 239.0,
  'hours': [22],
  'exposure': 310.7,
  'NormalizedScore': 81.2,
  'label': 'Best'},
 {'start_hour': 21,
  'end_hour': 22,
  'avg_aqi': 244.0,
  'hours': [21],
  'exposure': 317.2,
  'NormalizedScore': 82.9,
  'label': 'Acceptable'},
 {'start_hour': 20,
  'end_hour': 21,
  'avg_aqi': 249.0,
  'hours': [20],
  'exposure': 373.5,
  'NormalizedScore': 97.6,
  'label': 'Acceptable'},
 {'start_hour': 19,
  'end_hour': 20,
  'avg_aqi': 252.0,
  'hours': [19],
  'exposure': 378.0,
  'NormalizedScore': 98.8,
  'label': 'Avoid'},
 {'start_hour': 18,
  'end_hour': 19,
  'avg_aqi': 255.0,
  'hours': [18],
  'exposure': 382.5,
  'NormalizedScore': 100.0,
  'label': 'Avoid'}]

In [22]:
# Show top results
for w in windows[:3]:
    print(
        f"{w['start_hour']}:00–{w['end_hour']}:00 | "
        f"AQI {round(w['avg_aqi'])} | "
        f"Score {round(w['NormalizedScore'],1)} | "
        f"{w['label']}"
    )

23:00–24:00 | AQI 233 | Score 79.2 | Best
22:00–23:00 | AQI 239 | Score 81.2 | Best
21:00–22:00 | AQI 244 | Score 82.9 | Acceptable


In [23]:
import json
from pathlib import Path
from datetime import date

from src.penalties import (
    get_duration_factor,
    get_activity_factor,
    get_time_penalty,
)
from src.exposure import compute_exposure_score
from src.windows import generate_time_windows
from src.ranker import label_windows


JSON_PATH = Path("data/delhi_hourly_aqi.json")


with open(JSON_PATH, "r") as f:
    hourly_aqi = json.load(f)["data"]

today = date.today().strftime("%Y-%m-%d")

hourly_aqi = [
    h for h in hourly_aqi
    if h.get("date") == today
]

if not hourly_aqi:
    raise ValueError("No AQI data available for today.")


# User input
duration = 30  # minutes
activity = "walking"

duration_factor = get_duration_factor(duration)
activity_factor = get_activity_factor(activity)

windows = generate_time_windows(hourly_aqi, duration)



for w in windows:
    hour_penalties = [get_time_penalty(h) for h in w["hours"]]
    time_penalty = sum(hour_penalties) / len(hour_penalties)
    w["exposure"] = compute_exposure_score(
        w["avg_aqi"],
        duration_factor,
        activity_factor,
        time_penalty,
    )

    # w["NormalizedScore"] = min(100, w["exposure"] / 10)

max_exposure = max(w["exposure"] for w in windows)

for w in windows:
    w["NormalizedScore"] = round(
        (w["exposure"] / max_exposure) * 100, 1
    )

current_hour = datetime.now().hour

windows = [
    w for w in windows
    if w["start_hour"] >= current_hour
]

In [24]:
windows

[{'start_hour': 18,
  'end_hour': 19,
  'avg_aqi': 255.0,
  'hours': [18],
  'exposure': 382.5,
  'NormalizedScore': 99.2},
 {'start_hour': 19,
  'end_hour': 20,
  'avg_aqi': 252.0,
  'hours': [19],
  'exposure': 378.0,
  'NormalizedScore': 98.1},
 {'start_hour': 20,
  'end_hour': 21,
  'avg_aqi': 249.0,
  'hours': [20],
  'exposure': 373.5,
  'NormalizedScore': 96.9},
 {'start_hour': 21,
  'end_hour': 22,
  'avg_aqi': 244.0,
  'hours': [21],
  'exposure': 317.2,
  'NormalizedScore': 82.3},
 {'start_hour': 22,
  'end_hour': 23,
  'avg_aqi': 239.0,
  'hours': [22],
  'exposure': 310.7,
  'NormalizedScore': 80.6},
 {'start_hour': 23,
  'end_hour': 24,
  'avg_aqi': 233.0,
  'hours': [23],
  'exposure': 302.90000000000003,
  'NormalizedScore': 78.6}]

In [26]:
windows = sorted(windows, key=lambda x: x["exposure"])
windows = label_windows(windows)

# Show top results
for w in windows[:]:
    print(
        f"{w['start_hour']}:00–{w['end_hour']}:00 | "
        f"AQI {round(w['avg_aqi'])} | "
        f"Score {round(w['NormalizedScore'],1)} | "
        f"{w['label']}"
    )


23:00–24:00 | AQI 233 | Score 78.6 | Best
22:00–23:00 | AQI 239 | Score 80.6 | Best
21:00–22:00 | AQI 244 | Score 82.3 | Acceptable
20:00–21:00 | AQI 249 | Score 96.9 | Acceptable
19:00–20:00 | AQI 252 | Score 98.1 | Avoid
18:00–19:00 | AQI 255 | Score 99.2 | Avoid


In [33]:
best = windows[0]
# best = windows[0]
worst = windows[-1]

print("\n✅ BEST TIME TO GO OUT (FROM NOW)\n")

print(
    f"🕒 {best['start_hour']}:00 – {best['end_hour']}:00\n"
    f"Risk Level: {best['label']}\n"
    f"Exposure Score: {best['NormalizedScore']} / 100\n"
)

print("Why this window:")
print("• Lower relative pollution compared to rest of the remaining day")
print("• Time-of-day conditions are more favorable\n")

print("\n✅ WORST TIME TO GO OUT (FROM NOW)\n")

print(
    f"🕒 {worst['start_hour']}:00 – {worst['end_hour']}:00\n"
    f"Risk Level: {worst['label']}\n"
    f"Exposure Score: {worst['NormalizedScore']} / 100\n"
)


# Show comparison with worst upcoming window
worst = windows[-1]
risk_multiplier = round(
    worst["exposure"] / best["exposure"], 1
)

def risk_context(multiplier):
    if multiplier < 1.3:
        return "minor"
    elif multiplier < 2.0:
        return "moderate"
    else:
        return "severe"


context = risk_context(risk_multiplier)

if context == "minor":
    print(
        f"ℹ️ Risk later today is only slightly higher "
        f"(~{risk_multiplier}×)."
    )

elif context == "moderate":
    print(
        f"⚠️ Risk later today is ~{risk_multiplier}× higher.\n"
        "Short trips are still manageable with caution."
    )
    print("• Prefer shorter durations")
    print("• Avoid unnecessary exertion")

else:  # severe
    print(
        f"🚨 Risk later today is ~{risk_multiplier}× higher.\n"
        "Outdoor exposure is strongly discouraged."
    )
    print("• Postpone if possible")
    print("• If unavoidable, keep it under 10 minutes")
    print("• Avoid any brisk activity")



✅ BEST TIME TO GO OUT (FROM NOW)

🕒 23:00 – 24:00
Risk Level: Best
Exposure Score: 78.6 / 100

Why this window:
• Lower relative pollution compared to rest of the remaining day
• Time-of-day conditions are more favorable


✅ WORST TIME TO GO OUT (FROM NOW)

🕒 18:00 – 19:00
Risk Level: Avoid
Exposure Score: 99.2 / 100

⚠️ Risk later today is ~1.3× higher.
Short trips are still manageable with caution.
• Prefer shorter durations
• Avoid unnecessary exertion


In [31]:
current_hour = datetime.now().hour -1
current_hour

17

In [32]:
is_last_window = len(windows) == 1
is_last_window

False